In [2]:
from langgraph.graph import StateGraph,END
from typing import TypedDict,Annotated
import operator
from langchain_core.messages import AnyMessage,SystemMessage,HumanMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import ToolMessage

In [3]:
# 直接传入 tavily_api_key 参数
tool = TavilySearchResults(tavily_api_key="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", max_results=2)

# 或者设置环境变量并使用
import os
os.environ["TAVILY_API_KEY"] = "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"
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],operator.add]

In [5]:
# from langgraph.checkpoint.sqlite import SqliteSaver

# memory = SqliteSaver.from_conn_string(":memory:")
from langgraph.checkpoint.memory import MemorySaver

# 使用 MemorySaver 来代替 SqliteSaver
memory = MemorySaver()

In [6]:
class Agent:
    def __init__(self,model,tools,checkpointer,system=""):
        self.system=system
        graph=StateGraph(AgentState)
        graph.add_node("llm",self.call_openai)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True:"action",False:END}
        )
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile(checkpointer=checkpointer)
        self.tools={t.name:t for t in tools}
        self.model=model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        import threading
        print(f"take_action called in thread: {threading.current_thread().name}")
        tool_calls = state['messages'][-1].tool_calls
        results = []
        print(f"take_action called with tool_calls: {tool_calls}")
        for t in tool_calls:
            print(f"Calling: {t}") 
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"action {t['name']}, result: {result}")
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(base_url= "https://api.zhizengzeng.com/v1/",
                api_key= "sk-zk282c91e7660ee3aefc73a38060adc99596725965513925",
                  model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages=[HumanMessage(content="What is the weather in sf?")]

In [9]:
thread={"configurable":{"thread_id":"1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oX89qFMEULJYr76x73Hfpepc', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 151, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_62f96df568', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f036d3de-3627-43ab-897a-dd6b973446b6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_oX89qFMEULJYr76x73Hfpepc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 21, 'total_tokens': 172, 'input_token_details': {'audio': 0, 'cache

In [11]:
messages=[HumanMessage(content="What is the weather in la?")]
thread={"configurable":{"thread_id":"1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ln3pOy1OsLmmGNXVcm7fQh7N', 'function': {'arguments': '{"query":"Los Angeles weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 787, 'total_tokens': 808, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_62f96df568', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9e3a04ec-dcdd-4adb-8bbe-534c8cb0a856-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather today'}, 'id': 'call_Ln3pOy1OsLmmGNXVcm7fQh7N', 'type': 'tool_call'}], usage_metadata={'input_tokens': 787, 'output_tokens': 21, 'total_tokens': 808, 'input_token_details': {'audio': 0, 'cache_rea

In [12]:
messages=[HumanMessage(content="Which one is warmer")]
thread={"configurable":{"thread_id":"1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Currently, Los Angeles is slightly warmer than San Francisco. Los Angeles has a temperature of 12.2°C (54.0°F), while San Francisco has a temperature of 11.7°C (53.1°F).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 1588, 'total_tokens': 1635, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1408}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_62f96df568', 'finish_reason': 'stop', 'logprobs': None}, id='run-daab5be2-480c-40c6-b36d-0330255230c7-0', usage_metadata={'input_tokens': 1588, 'output_tokens': 47, 'total_tokens': 1635, 'input_token_details': {'audio': 0, 'cache_read': 1408}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [13]:
messages=[HumanMessage(content="Which one is warmer")]
thread={"configurable":{"thread_id":"2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please specify the locations or items you are comparing to determine which one is warmer?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 148, 'total_tokens': 167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_62f96df568', 'finish_reason': 'stop', 'logprobs': None}, id='run-d75a63a5-6041-472a-8e54-edd98df89d54-0', usage_metadata={'input_tokens': 148, 'output_tokens': 19, 'total_tokens': 167, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [65]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

# 使用适合 SQLite 的内存数据库连接字符串
memory=AsyncSqliteSaver.from_conn_string(':memory:')
abot = Agent(model, [tool], system=prompt, checkpointer=memory)


In [67]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

AttributeError: '_AsyncGeneratorContextManager' object has no attribute 'get_next_version'